In [1]:
try:
    import graphene
    import flask
    import flask_graphql
except ImportError:
    !/opt/conda/envs/python2/bin/pip install graphene Flask-GraphQL Flask

import json
import graphene
from os import getenv
from idr_graphql import schema
from omero.gateway import BlitzGateway

No handlers could be found for logger "omero.gateway"


In [2]:
def omero_connect():
    conn = BlitzGateway(
        host=getenv('IDR_HOST'),
        username=getenv('IDR_USER'),
        passwd=getenv('IDR_PASSWORD'),
        secure=True)
    r = conn.connect()
    assert r
    return conn

c = omero_connect()
ctx = dict(qs=c.getQueryService())

In [3]:
#print(schema.execute('{gene(key: "Gene Symbol", value: "polo") {id}}', context=ctx).data)
print(schema.execute('{image(id: 1030631) {annotations(mapr: GENE) {name value {name value}}}}', context=ctx).data)

OrderedDict([('image', OrderedDict([('annotations', [OrderedDict([('name', 'polo'), ('value', [OrderedDict([('name', 'Gene Identifier'), ('value', 'CG12306')]), OrderedDict([('name', 'Gene Identifier URL'), ('value', 'http://www.flybase.org/cgi-bin/uniq.html?field=SYN&db=fbgn&context=CG12306')]), OrderedDict([('name', 'Gene Symbol'), ('value', 'polo')])])])])]))])


In [4]:
# TODO
from promise import Promise
from promise.dataloader import DataLoader

class ImageLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        return Promise.resolve([get_user(id=key) for key in keys])


def qlbuild(*args):
    q = ' '.join(qlbuild(*arg) if isinstance(arg, list) else arg
                 for arg in args)
    return '{' + q + '}'


def execprint(*args):
    if len(args) > 1:
        q = qlbuild(*args)
    else:
        q = args[0]
    print('query: ' + q)
    s = schema.execute(q, context=ctx)
    if s.errors:
        print(s.errors)
    else:
        print(json.dumps(s.data, indent=2))

#print(schema.execute('{geneSymbols(symbol: "polo") {id}}').data)
#print(schema.execute('{ geneSymbols(symbol: "polo") {id images {id}}}').data)
execprint('image(id: 1030631)', ['id'])
execprint('image(id: 1030631)', ['id', 'name', 'annotations',
                                 ['__typename', 'id', 'name', 'value',
                                  ['name', 'value']]])

# execprint('image(id: 1030631)', ['id', 'name', 'annotations',
#     ['__typename', 'id', 'name', 'images', ['id', 'name']]])

query: {image(id: 1030631) {id}}
{
  "image": {
    "id": 1030631
  }
}
query: {image(id: 1030631) {id name annotations {__typename id name value {name value}}}}
{
  "image": {
    "id": 1030631, 
    "name": "Primary_062 [Well B11 Field #1]", 
    "annotations": [
      {
        "__typename": "Phenotype", 
        "id": 6672118, 
        "name": "defective cell-cell aggregation", 
        "value": [
          {
            "name": "Phenotype", 
            "value": "defective cell-cell aggregation"
          }, 
          {
            "name": "Phenotype Term Name", 
            "value": "fewer aggregated cells in population phenotype"
          }, 
          {
            "name": "Phenotype Term Accession", 
            "value": "CMPO_0000416"
          }, 
          {
            "name": "Phenotype Term Accession URL", 
            "value": "http://www.ebi.ac.uk/cmpo/CMPO_0000416"
          }
        ]
      }, 
      {
        "__typename": "Gene", 
        "id": 6672872, 
       

In [5]:
execprint('gene(key: "Gene Symbol", value: "polo")', ['id', 'name', 'images',
    ['id', 'name', 'annotations(mapr: PHENOTYPE)', ['id', 'name']]])

query: {gene(key: "Gene Symbol", value: "polo") {id name images {id name annotations(mapr: PHENOTYPE) {id name}}}}
{
  "gene": [
    {
      "id": 6672872, 
      "name": "polo", 
      "images": [
        {
          "id": 1030631, 
          "name": "Primary_062 [Well B11 Field #1]", 
          "annotations": [
            {
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 1030633, 
          "name": "Primary_062 [Well B11 Field #3]", 
          "annotations": [
            {
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 1030632, 
          "name": "Primary_062 [Well B11 Field #2]", 
          "annotations": [
            {
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 931327, 
  

In [6]:
c.close()